In [72]:
import requests
import pandas as pd
from sqlalchemy import create_engine, text
import time
import urllib.parse

In [73]:
# Sua chave de API Alpha Vantage
api_key = 'Y4E1GMWHA4EWZRHY'

# Lista de 10 símbolos de ações importantes
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'JPM', 'V', 'DIS', 'FB']

In [74]:
# Função para buscar dados de ações
def get_stock_data(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=60min&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if 'Time Series (60min)' in data:
        time_series = data['Time Series (60min)']
        df = pd.DataFrame(time_series).T
        df = df.reset_index().rename(columns={'index': 'timestamp'})
        df['symbol'] = symbol
        return df
    else:
        print(f"Erro ao buscar dados para {symbol}")
        return pd.DataFrame()

In [75]:
# Função para obter dados de várias ações e consolidar em um dataframe
def fetch_all_data(symbols):
    all_data = pd.DataFrame()
    for symbol in symbols:
        stock_data = get_stock_data(symbol)
        if not stock_data.empty:
            all_data = pd.concat([all_data, stock_data])
        time.sleep(12)  # Respeitar o limite de requisições da API
    return all_data

In [76]:
# Parâmetros de conexão
usuario = "root"
senha = "Are@0051"
host = "localhost"  
nome_do_banco = "shares"
senha_codificada = urllib.parse.quote_plus(senha)

conexao_string = f"mysql+pymysql://{usuario}:{senha_codificada}@{host}/{nome_do_banco}"

engine = create_engine(conexao_string)

In [77]:
# Função para inserir os dados de forma incremental com 'ON DUPLICATE KEY UPDATE'
def upsert_to_mysql(df):
    if df.empty:
        print("Nenhum dado para inserir.")
        return

    # Conectando ao MySQL
    with engine.connect() as connection:
        for _, row in df.iterrows():
            query = text(f"""
                INSERT INTO stock_prices (timestamp, open, high, low, close, volume, symbol)
                VALUES ('{row['timestamp']}', {row['1. open']}, {row['2. high']}, {row['3. low']}, {row['4. close']}, {row['5. volume']}, '{row['symbol']}')
                ON DUPLICATE KEY UPDATE
                open = VALUES(open),
                high = VALUES(high),
                low = VALUES(low),
                close = VALUES(close),
                volume = VALUES(volume);
            """)
            connection.execute(query)

Erro ao buscar dados para TSLA
Erro ao buscar dados para NVDA
Erro ao buscar dados para JPM
Erro ao buscar dados para V
Erro ao buscar dados para DIS
Erro ao buscar dados para FB
Dados atualizados com sucesso!
